In [1]:
from icepyx import icesat2data as ipd
import os, glob, re, h5py, sys, pyproj
import matplotlib as plt
import shutil
import numpy as np
from pprint import pprint
from astropy.time import Time
from scipy.signal import correlate, detrend
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget
import pointCollection as pc

In [2]:
### Where are the data to be processed
#datapath = '/home/jovyan/shared/surface_velocity/FIS_ATL06'

#local data path
datapath = '/media/rag110/ADATA SD700/ICESat2/download/FIS'

ATL06_files=glob.glob(os.path.join(datapath, '*.h5'))


### Where to save the results
#out_path = '/home/jovyan/shared/surface_velocity/ATL06_out/'


#local out_path

out_path = '/media/rag110/ADATA SD700/ICESat2/output/FIS/'

#Measures spatial extent and paths to tif files
spatial_extent = np.array([-65, -86, -55, -81])

#Rodrigo Gomez Fell computer path @ UC
vel_x_path = '/mnt/user1/Antarctica/Quantarctica3/Glaciology/MEaSUREs Ice Flow Velocity/anta_phase_map_VX.tif'
vel_y_path = '/mnt/user1/Antarctica/Quantarctica3/Glaciology/MEaSUREs Ice Flow Velocity/anta_phase_map_VY.tif'



In [3]:
### Revised version of code from Ben Smith to read in the hdf5 files and extract necessary datasets and information
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    # below: file_re = regular expression, it will pull apart the regular expression to get the information from the filename
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
        D['data_start_utc'] = h5f['/ancillary_data/data_start_utc'][:]
        D['delta_time'] = h5f['/' + beam + '/land_ice_segments/delta_time'][:]
        D['segment_id'] = h5f['/' + beam + '/land_ice_segments/segment_id'][:]
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

In [4]:
def add_surface_velocity_to_is2_dict(x_ps_beam, y_ps_beam , spatial_extent, vel_x_path, vel_y_path):
    """
    
    is2_dict: Python dictionary with ATL06 track data
    spatial_extent: bounding box of the interest area in the format:
                    (e.g. [-65, -86, -55, -81] == [min_lon, min_lat, max_lon, max_lat])
    path: local path to velocity data
    vel_x: tif velocity raster with x component
    vel_y: tif velocity raster with y component
    
    """
    
    #fix with if statement about type of list or array DONE
    
    if type(spatial_extent) == type([]):
    
        spatial_extent = np.array([spatial_extent])
    
        
    lat=spatial_extent[[1, 3, 3, 1, 1]]
    lon=spatial_extent[[2, 2, 0, 0, 2]]

    # project the coordinates to Antarctic polar stereographic
    xy=np.array(pyproj.Proj(3031)(lon, lat))
    # get the bounds of the projected coordinates 
    XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
    YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
    
    #Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,vel_x), bounds=[XR, YR])
    #Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,vel_y), bounds=[XR, YR])
    
    Measures_vx=pc.grid.data().from_geotif(vel_x_path, bounds=[XR, YR])
    Measures_vy=pc.grid.data().from_geotif(vel_y_path, bounds=[XR, YR])
    
    vx = Measures_vx.interp(x_ps_beam,y_ps_beam)
    vy = Measures_vy.interp(x_ps_beam,y_ps_beam)

    #Solve for angle to rotate Vy to be along track and Vx to be across track
    import math
    xL=abs((x_ps_beam[0])-(x_ps_beam[1]))
    yL=abs((y_ps_beam[0])-(y_ps_beam[1]))

    #decides if is descending or ascending path
    if x_ps_beam[0]-x_ps_beam[1] < 0:

        theta_rad=math.atan(xL/yL)
        #theta_deg=theta_rad*180/math.pi
        v_along=vy/math.cos(theta_rad)
        #v_across=vx/math.cos(theta_rad)

    else:
    
        theta_rad=math.atan(xL/yL)
        #theta_deg=theta_rad*180/math.pi
        v_along=vy/math.sin(theta_rad)
        #v_across=vx/math.sin(theta_rad)
    
    #Vdiff=vy-v_along
    return v_along

In [6]:
filename = '/media/rag110/ADATA SD700/ICESat2/download/FIS/processed_ATL06_20190213005328_07110211_003_01.h5'
beam = 'gt1r'
D = atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=3031)



In [10]:
vel = add_surface_velocity_to_is2_dict(D['x'], D['y'] , spatial_extent, vel_x_path, vel_y_path)